In [2]:
import transformers

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
transformers.BERT_PRETRAINED_MODEL_ARCHIVE_MAP["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/pytorch_model.bin"
transformers.BERT_PRETRAINED_CONFIG_ARCHIVE_MAP["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/config.json"
transformers.tokenization_bert.PRETRAINED_VOCAB_FILES_MAP["vocab_file"]["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/vocab.txt"
transformers.tokenization_bert.PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES["bert-base-finnish-cased"]=512
transformers.tokenization_bert.PRETRAINED_INIT_CONFIGURATION["bert-base-finnish-cased"]={'do_lower_case': False}


In [4]:
model=transformers.BertForMaskedLM.from_pretrained("bert-base-finnish-cased")

In [5]:
model=model.cuda()

In [5]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-finnish-cased")

In [7]:
CLS,SEP,MASK=tokenizer.convert_tokens_to_ids(["[CLS]","[SEP]","[MASK]"])
texts=["Minulla on koira joka on ruskea ja vihainen"]
tokenized_texts=[tokenizer.tokenize(t) for t in texts]
tokenized_texts

[['Minulla', 'on', 'koira', 'joka', 'on', 'ruskea', 'ja', 'vihainen']]

In [8]:
texts_ids=[torch.tensor([CLS]+tokenizer.convert_tokens_to_ids(tt)) for tt in tokenized_texts]
texts_ids=torch.nn.utils.rnn.pad_sequence(texts_ids,batch_first=True,padding_value=MASK)
texts_ids=texts_ids.cuda()
resp=model(texts_ids)

In [9]:
print(texts_ids.shape)
print(resp[0].shape)

torch.Size([1, 9])
torch.Size([1, 9, 50105])


In [10]:

new_texts

NameError: name 'new_texts' is not defined

In [ ]:
min_lead_in=3
s_len=len(texts_ids[0])
to_predict=s_len-min_lead_in-1
assert(to_predict)>0
new_texts=texts_ids[0].unsqueeze(0).expand((to_predict,-1))
lower_triangle=torch.tril(torch.ones((to_predict,to_predict+1)))
mask=torch.cat((torch.ones((to_predict,min_lead_in)),lower_triangle),-1)
mask=mask.cuda()
print(mask)

In [ ]:
out=model(input_ids=new_texts,attention_mask=mask)
out

In [ ]:
out=model(input_ids=new_texts,attention_mask=mask,masked_lm_labels=new_texts.reshape((-1,)))
loss,pred=out
loss
#out_reshaped=out[0].view(-1,model.config.vocab_size)
#gold=new_texts.reshape((-1,))
#torch.nn.CrossEntropyLoss()(out_reshaped,gold)

In [ ]:
print("texts",new_texts)
filter=torch.cat((torch.zeros((5,4),dtype=torch.long),torch.eye(5,dtype=torch.long)),-1).cuda()
print("filter",filter)
gold=filter*(new_texts+1)-1
print("gold",gold)

In [ ]:
gold=filter*(new_texts+1)-1
out=model(input_ids=new_texts,attention_mask=mask,masked_lm_labels=gold)
loss,pred=out
loss